# 加速：多線程爬蟲



* 了解知乎 API 使用方式與回傳內容
* 撰寫程式存取 API 且添加標頭

## 作業目標

* 找一個之前實作過的爬蟲改用多線程改寫，比較前後時間的差異。





In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import _thread

In [2]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
#     for p in news_content:
#         """
#         .string屬性說明：
#         (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
#         (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
#         (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
#         """
#         if ((p.string) is not None):
#             print(p.string)

In [3]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

browser = webdriver.Chrome(executable_path='./Data/chromedriver')
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("return document.body.scrollHeight;")

while True:
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)
    current_height = browser.execute_script("return document.body.scrollHeight;")
    
    if current_height == last_height:
        print('到達頁面底端')
        break
    else:
        print('網頁更新中...')
        last_height = current_height
        continue
        
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})
news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

# 關閉瀏覽器
browser.quit();

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面底端


In [4]:
# 單線程
startTime = time.time()
for i, news_item in enumerate(news_block):
#     print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
#     print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)
    finishTime = time.time()
print(finishTime - startTime)

15.229621887207031


In [5]:
# 多線程
startTime = time.time()
for i, news_item in enumerate(news_block):
#     print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
#     print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    _thread.start_new_thread(getNewsDetailContent, (externalLink, ))
    finishTime = time.time()
print(finishTime - startTime)

0.017989158630371094
